In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data Preparation

In [0]:
import pandas as pd
df_promo=pd.read_csv('/content/drive/My Drive/wikipedia-promotional-articles/promotional.csv')

In [0]:
df_promo.head()

,text,advert,coi,fanpov,pr,resume,url
0,"1 Litre no Namida 1, lit. 1 Litre of Tears als...",0,0,1,0,0,https://en.wikipedia.org/wiki/1%20Litre%20no%2...
1,"1DayLater was free, web based software that wa...",1,1,0,0,0,https://en.wikipedia.org/wiki/1DayLater
2,1E is a privately owned IT software and servic...,1,0,0,0,0,https://en.wikipedia.org/wiki/1E
3,1Malaysia pronounced One Malaysia in English a...,1,0,0,0,0,https://en.wikipedia.org/wiki/1Malaysia
4,"The Jerusalem Biennale, as stated on the Bienn...",1,0,0,0,0,https://en.wikipedia.org/wiki/1st%20Jerusalem%...


In [0]:
df_good=pd.read_csv('/content/drive/My Drive/wikipedia-promotional-articles/good.csv')

In [0]:
len(df_good)

30279

In [0]:
len(df_promo)

23837

In [0]:
df_good.head()

,text,url
0,Nycticebus linglom is a fossil strepsirrhine p...,https://en.wikipedia.org/wiki/%3F%20Nycticebus...
1,Oryzomys pliocaenicus is a fossil rodent from ...,https://en.wikipedia.org/wiki/%3F%20Oryzomys%2...
2,.hack dt hk is a series of single player actio...,https://en.wikipedia.org/wiki/.hack%20%28video...
3,The You Drive Me Crazy Tour was the second con...,https://en.wikipedia.org/wiki/%28You%20Drive%2...
4,0 8 4 is the second episode of the first seaso...,https://en.wikipedia.org/wiki/0-8-4


In [0]:
import numpy as np
random_indices=np.random.choice(len(df_good),replace=False,size=len(df_promo))
df_good=df_good.iloc[random_indices]

In [0]:
len(df_good)

23837

In [0]:
!pip install transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 542kB 3.4MB/s 
     |████████████████████████████████| 3.7MB 16.0MB/s 
     |████████████████████████████████| 1.0MB 49.4MB/s 
     |████████████████████████████████| 870kB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=6d99181fa87af8ba09eed3b0fbe988f7df954cf97a83337dedd6277371be8832
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
# import nltk
# nltk.download('punkt')
import pickle
df_promo.drop(labels=['advert','coi','fanpov','pr','resume','url'],inplace=True,axis=1)
df_promo['label']=1
df_good.drop(labels=['url'],axis=1,inplace=True)
df_good['label']=0

df_good['encoding']=df_good['text'].apply(lambda x: tokenizer.tokenize(x))
df_promo['encoding']=df_promo['text'].apply(lambda x: tokenizer.tokenize(x))
df=pd.concat([df_good,df_promo],axis=0,join='inner')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
import pickle
import torch.nn as nn
def pad(tokens):
  if(len(tokens)<512):
    return tokens+['[PAD]' for _ in range(512-len(tokens))]
  else:
    return tokens[:512]  
    
df['encoding']=df['encoding'].apply(lambda x: pad(x))
print(df.isna().sum())

df['encoding']=df['encoding'].apply(lambda x:tokenizer.convert_tokens_to_ids(x))
df.head()

text        0
label       0
encoding    0
dtype: int64


,text,label,encoding
24627,The Siege of Berat in Albania by the forces of...,0,"[1996, 6859, 1997, 2022, 8609, 1999, 10407, 20..."
12005,Eric Harrison 10 August 1886 5 September 1945 ...,0,"[4388, 6676, 2184, 2257, 6929, 1019, 2244, 338..."
20978,Papers is a song by American recording artist ...,0,"[4981, 2003, 1037, 2299, 2011, 2137, 3405, 306..."
25415,The United States Coast Guard USCG Women's Res...,0,"[1996, 2142, 2163, 3023, 3457, 15529, 2290, 23..."
675,The 1973 Preakness Stakes was the 98th running...,0,"[1996, 3381, 3653, 4817, 2791, 7533, 2001, 199..."


In [0]:
import pickle
# with open('/content/drive/My Drive/wikipedia-promotional-articles/train_GRU_Pool.pkl','wb') as f:
#   pickle.dump(df,f)

with open('/content/drive/My Drive/wikipedia-promotional-articles/train.pkl','rb') as f:
  df=pickle.load(f)
df.head()

,text,label,encoding,attn_mask,cls_index
9287,Eppur Si Muove is the 104th episode of The Wes...,0,"[101, 4958, 5311, 9033, 14163, 21818, 2003, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 24, 36, 59, 98, 120, 172, 204, 245, 257, 2..."
3911,USS Benham Destroyer No. 49 DD 49 was an Aylwi...,0,"[101, 7234, 3841, 3511, 9799, 2053, 1012, 102,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 8, 36, 63, 86, 131, 153, 185, 216, 232, 27..."
22101,Prison Break is an American television serial ...,0,"[101, 3827, 3338, 2003, 2019, 2137, 2547, 7642...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 19, 73, 98, 146, 172, 198, 221, 238, 266, ..."
17770,Narinder Kumar Mehra born 4 November 1949 is a...,0,"[101, 6583, 6657, 4063, 9600, 2033, 13492, 214...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 44, 65, 75, 125, 180, 200, 286, 358, 383, ..."
2963,"Keith Aulie born June 11, 1989 is a Canadian p...",0,"[101, 6766, 8740, 8751, 2141, 2238, 2340, 1010...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 34, 54, 88, 118, 146, 162, 191, 227, 250, ..."


In [0]:
from torch.utils.data import Dataset
class ClassifierDataset(Dataset):
  def __init__(self,df):
    self.df=df

  def __len__(self): 
    return len(self.df)

  def __getitem__(self,index):
    return torch.tensor(self.df.iloc[index]['encoding']),torch.tensor(self.df.iloc[index]['label'])

In [0]:
from torch.utils.data import DataLoader
import numpy as np
msk = np.random.rand(len(df)) < 0.8
train=df[msk]
val=df[~msk]
train_set=ClassifierDataset(train)
val_set=ClassifierDataset(val)
train_loader=DataLoader(train_set, batch_size = 16)
val_loader = DataLoader(val_set, batch_size = 16)
import torch
import torch.nn as nn

# Model

In [0]:
class DocumentClassifier(nn.Module):
    def __init__(self, embedding_size,hidden_size):
        super(DocumentClassifier, self).__init__()
        self.embedding=nn.Embedding(tokenizer.vocab_size,embedding_size,padding_idx=tokenizer.pad_token_id)
        self.gru=nn.GRUCell(embedding_size,hidden_size)
        self.maxpool=nn.MaxPool2d((3,3),stride=2)
        self.linear=nn.Linear(49,1)
        self.out=nn.Linear(255,1)
        
    def forward(self, seq):
        embedded=self.embedding(seq)
        hidden=None
        forward_output=[]
        for i in range(embedded.shape[1]):
            hidden=self.gru(embedded[:,i,:],hidden)            
            forward_output.append(hidden.unsqueeze(1))
        forward_output=torch.cat((forward_output),1)        
        
        backward_output=[]
        hidden=None
        for i in reversed(range(embedded.shape[1])):
            hidden=self.gru(embedded[:,i,:],hidden)
            backward_output.append(hidden.unsqueeze(1))
        backward_output=torch.cat((backward_output),1)

        output=torch.cat((forward_output,backward_output),2)        
        output=self.maxpool(output)
        output=self.linear(output)
        logits=self.out(output.squeeze())
        return logits

# Training

In [0]:
import torch.optim as optim
import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_size=100
hidden_size=50
model=DocumentClassifier(embedding_size,hidden_size).to(device)
if(os.path.exists('/content/drive/My Drive/wikipedia-promotional-articles/model_GRU_Pool.pt')):
    model.load_state_dict(torch.load('/content/drive/My Drive/wikipedia-promotional-articles/model_GRU_Pool.pt'))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-6)

sig=nn.Sigmoid()
# training_loss=[]
val_losses=[]
for _e in range(50):
    train_loss=0
    for t, (seq, labels) in enumerate(train_loader):
        seq=seq.to(device)
        labels =labels.to(device)                 
        optimizer.zero_grad()
        logits=model(seq)       
        loss = criterion(logits.squeeze(-1), labels.float().view(-1))
        train_loss+=loss.data.item()
        loss.backward()
        optimizer.step()
    train_loss= np.mean(train_loss)
    val_loss=0
    for t, (seq, labels) in enumerate(val_loader):       
        seq=seq.to(device)
        labels =labels.to(device)                
        optimizer.zero_grad()
        logits=model(seq)
        loss = criterion(logits.squeeze(-1), labels.float())
        val_loss+=loss.data.item()
    val_loss= np.mean(val_loss)   
    if(len(val_losses)>0 and val_loss<min(val_losses)):
      torch.save(model.state_dict(), '/content/drive/My Drive/wikipedia-promotional-articles/model_GRU_Pool.pt')  
    val_losses.append(val_loss)      
    print('training loss:{} validation loss:{}'.format(train_loss,val_loss))      

training loss:687.6087683381629 validation loss:186.13017386151478
training loss:609.9301257077605 validation loss:166.96697031136137
training loss:555.9381862428272 validation loss:155.46413249499165
training loss:523.9732703529298 validation loss:148.56438296381384
training loss:504.9015784121584 validation loss:144.35515769082122
training loss:493.1696209271904 validation loss:141.70349595858715
training loss:485.56245479756035 validation loss:139.9535783934407


In [0]:
with open('/content/drive/My Drive/wikipedia-promotional-articles/val_losses.pkl','wb') as f:
  pickle.dump(val_losses,f)

# Testing

In [0]:
import torch.optim as optim
import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_size=100
hidden_size=50
model=DocumentClassifier(embedding_size,hidden_size).to(device)
if(os.path.exists('/content/drive/My Drive/wikipedia-promotional-articles/model_GRU_Pool.pt')):
    model.load_state_dict(torch.load('/content/drive/My Drive/wikipedia-promotional-articles/model_GRU_Pool.pt'))
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr = 1e-4)

sig=nn.Sigmoid()
# training_loss=[]
val_losses=[]
probabs=[]
for seq, labels in val_loader:       
    seq=seq.to(device)
    labels =labels.to(device)
    logits=model(seq)
    out=sig(logits.view(-1))
    probabs.append(out) 
# print('training loss:{} validation loss:{}'.format(train_loss,val_loss))      

In [0]:
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(list(df['encoding']), list(df['label']), 
#                                                             random_state=2018, test_size=0.1)
# # Do the same for the masks.
# train_masks, validation_masks, _, _ = train_test_split(list(df['attention_mask']), list(df['label']),
#                                              random_state=2018, test_size=0.1)
with open('/content/drive/My Drive/wikipedia-promotional-articles/probabs.pkl','wb') as f:
  pickle.dump(probabs,f)